In [1]:
import numpy as np
import pandas as pd
import bob.learn.em
from sklearn import preprocessing
from tqdm import tqdm
from datetime import datetime

In [2]:
data = pd.read_csv('./list_mfccs.csv')

In [3]:
data.shape

(1700000, 42)

In [4]:
data.head()

,filename,class,lbl,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,...,mfcc_30,mfcc_31,mfcc_32,mfcc_33,mfcc_34,mfcc_35,mfcc_36,mfcc_37,mfcc_38,mfcc_39
0,blues/blues.00000.au,blues,1,-391.650249,82.601436,-12.222550,29.772965,6.642475,22.672382,-28.166884,...,0.906655,0.034052,-0.086309,-0.93997,-1.438514,-0.074889,0.188007,163126.478866,201.7571,34.271484
1,blues/blues.00000.au,blues,1,-368.206425,102.445621,-11.593758,50.964616,12.346113,25.832453,-17.689354,...,0.906655,0.034052,-0.086309,-0.93997,-1.438514,-0.074889,0.188007,150602.779701,201.7571,34.271484
2,blues/blues.00000.au,blues,1,-375.919845,96.091969,-19.516882,53.242674,13.998896,22.207447,-22.102237,...,0.906655,0.034052,-0.086309,-0.93997,-1.438514,-0.074889,0.188007,155723.291247,201.7571,34.271484
3,blues/blues.00000.au,blues,1,-365.078188,104.475720,-15.919827,44.830101,-0.103477,22.446645,-23.573937,...,0.906655,0.034052,-0.086309,-0.93997,-1.438514,-0.074889,0.188007,148013.441521,201.7571,34.271484
4,blues/blues.00000.au,blues,1,-360.093953,87.034275,-7.802828,60.510771,8.696025,38.717838,-4.617636,...,0.906655,0.034052,-0.086309,-0.93997,-1.438514,-0.074889,0.188007,143671.423459,201.7571,34.271484


In [7]:
cols = ['mfcc_%s'%i for i in range(1, 40)]
scaler = preprocessing.StandardScaler()
scaled_df = scaler.fit_transform(data[cols])

In [8]:
data_scaled = data.copy()
data_scaled[cols] = pd.DataFrame(scaled_df)

In [9]:
data_scaled.head()

,filename,class,lbl,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,...,mfcc_30,mfcc_31,mfcc_32,mfcc_33,mfcc_34,mfcc_35,mfcc_36,mfcc_37,mfcc_38,mfcc_39
0,blues/blues.00000.au,blues,1,-0.724054,-0.500629,-0.165197,-0.224717,0.344881,0.522743,-1.350355,...,0.714399,0.028203,-0.075622,-0.863715,-1.345201,-0.072421,0.18751,0.406762,0.196529,-0.217599
1,blues/blues.00000.au,blues,1,-0.525030,-0.067544,-0.145980,0.647703,0.637306,0.695705,-0.702556,...,0.714399,0.028203,-0.075622,-0.863715,-1.345201,-0.072421,0.18751,0.268448,0.196529,-0.217599
2,blues/blues.00000.au,blues,1,-0.590512,-0.206208,-0.388126,0.741486,0.722044,0.497295,-0.975393,...,0.714399,0.028203,-0.075622,-0.863715,-1.345201,-0.072421,0.18751,0.325000,0.196529,-0.217599
3,blues/blues.00000.au,blues,1,-0.498473,-0.023238,-0.278193,0.395156,-0.000983,0.510387,-1.066385,...,0.714399,0.028203,-0.075622,-0.863715,-1.345201,-0.072421,0.18751,0.239851,0.196529,-0.217599
4,blues/blues.00000.au,blues,1,-0.456160,-0.403885,-0.030121,1.040700,0.450166,1.400972,0.105636,...,0.714399,0.028203,-0.075622,-0.863715,-1.345201,-0.072421,0.18751,0.191897,0.196529,-0.217599


In [67]:
len(files)

850

In [68]:
files = data.filename.unique()
data_sample = data_scaled[data_scaled.filename.isin(files[:100])]

In [69]:
data_sample[data_sample.filename == 'blues/blues.00000.au'].shape

(2000, 42)

In [ ]:
mfcc_len = 39
gmm_len = 500
cols_tmp = cols[:mfcc_len]

prior_gmm = bob.learn.em.GMMMachine(gmm_len, mfcc_len)
prior_gmm.means = np.vstack(np.random.normal(0.3, 0.7, (gmm_len, mfcc_len)))
prior_gmm.variances = np.ones((gmm_len, mfcc_len)) * 0.5
weights = np.ones(gmm_len) / gmm_len
prior_gmm.weights = weights

gmm_stats_per_class = []
for idx in (range(len(data_sample))):
    if idx % 1000 == 0:
        print(idx)
    i = list(data_sample.loc[idx, cols_tmp])
    gmm_stats_container = bob.learn.em.GMMStats(gmm_len, mfcc_len)
    prior_gmm.acc_statistics(i, gmm_stats_container)
    gmm_stats_per_class.append(gmm_stats_container)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000


In [16]:
a = gmm_stats_per_class[3]
a.sum_pxx, a.sum_px, a.t

(array([[  8.53795181e-05,   1.85558757e-07,   2.65926876e-05,
           5.36547379e-05,   3.31971616e-10,   8.95096180e-05,
           3.90749081e-04,   6.10991614e-06,   1.66321199e-04,
           3.20340775e-04,   7.07021874e-05,   1.53451170e-04,
           6.57885661e-04,   2.17053145e-05,   5.82326422e-05,
           1.84412196e-04,   9.82704429e-05,   8.84459744e-04,
           2.45823807e-04,   4.28080014e-04,   6.05235803e-05,
           4.34315692e-05,   3.57126773e-05,   2.35858898e-04,
           2.21370685e-04,   3.69205118e-07,   3.87861125e-04,
           8.78456419e-04,   1.38524295e-04,   1.75368730e-04,
           2.73317020e-07,   1.96499275e-06,   2.56336878e-04,
           6.21790920e-04,   1.80218167e-06,   1.20814929e-05,
           1.97676089e-05,   1.32715979e-05,   1.62698212e-05],
        [  2.48084398e-01,   5.39171849e-04,   7.72694793e-02,
           1.55902770e-01,   9.64598775e-07,   2.60085091e-01,
           1.13538648e+00,   1.77533781e-02,   4.83273

In [17]:
a.shape

(5, 39)

In [65]:
subspace_dimension_of_t = 100
ivector_trainer = bob.learn.em.IVectorTrainer(update_sigma=True)
ivector_machine = bob.learn.em.IVectorMachine(
     prior_gmm, subspace_dimension_of_t, 10e-5)

In [66]:
# train IVector model

start_time = datetime.now()

bob.learn.em.train(ivector_trainer, ivector_machine,
                    gmm_stats_per_class, 50)

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))




Duration: 0:00:40.680855


In [ ]:
2:4 20:14 50:13 100:40

In [49]:
len(gmm_stats_per_class)

2000

In [50]:
ivector_machine(gmm_stats_per_class[0])

array([ 1.92646994,  0.71522319])

In [51]:
# Printing the session offset w.r.t each Gaussian component
print(ivector_machine.t)

[[  1.94462308e-01  -9.48188245e-01]
 [  6.47917673e-02  -5.07947823e-02]
 [ -6.11494189e-01   8.55452955e-01]
 [ -3.10107650e-01   5.68507147e-01]
 [ -5.84907993e-01   1.04086665e+00]
 [  1.70707054e-01  -2.51385248e-01]
 [ -8.55684926e-01   1.29568652e+00]
 [  2.38272373e-01  -4.33057652e-01]
 [ -2.32323417e-01   5.94197851e-01]
 [  5.15582828e-01  -6.25783811e-01]
 [ -8.20189608e-02   3.84779241e-01]
 [ -9.30479883e-01   4.98444729e-01]
 [ -3.52227196e-01   4.62483404e-02]
 [ -1.37574376e-01   1.85743177e-02]
 [ -1.75905115e-01   2.11383964e-01]
 [ -1.03768601e+00   1.08524138e+00]
 [  3.71124646e-01  -6.82599345e-02]
 [  1.06628266e-01   3.11221153e-02]
 [ -6.50093137e-01   6.76881920e-01]
 [ -2.98579506e-01   1.55062318e-01]
 [ -3.65300742e-01   4.15945819e-01]
 [  1.55466711e-01  -3.10553587e-01]
 [ -4.04804588e-01   4.52063213e-01]
 [  2.44032903e-01  -2.25565687e-01]
 [  2.70713756e-01   5.53757669e-01]
 [ -6.93011384e-01   6.75722330e-01]
 [  6.44632371e-01  -9.00416514e-01]
 

In [52]:
ivector_machine.t.shape

(195, 2)

In [54]:
ivector_machine.shape

(5, 39, 2)

In [55]:
5*39

195

In [247]:
prior_gmm.weights

array([ 0.3,  0.7])

In [244]:
np.random.seed(10)
# Generating some fake data
data_class1 = np.random.normal(0, 0.5, (10, 3))
data_class2 = np.random.normal(-0.2, 0.2, (10, 3))
data_tmp = [data_class1, data_class2]

# Creating a fake prior with 2 gaussians of dimension 3
prior_gmm = bob.learn.em.GMMMachine(2, 3)
prior_gmm.means = np.vstack((np.random.normal(0, 0.5, (1, 3)),
                               np.random.normal(1, 0.5, (1, 3))))
# All nice and round diagonal covariance
prior_gmm.variances = np.ones((2, 3)) * 0.5
prior_gmm.weights = np.array([0.3, 0.7])

# The input the the TV Training is the statistics of the GMM
gmm_stats_per_class = []
for d in data_tmp:
    for i in d:
        print(i)
        gmm_stats_container = bob.learn.em.GMMStats(2, 3)
        prior_gmm.acc_statistics(i, gmm_stats_container)
        print(gmm_stats_container.shape)
        gmm_stats_per_class.append(gmm_stats_container)

# # Finally doing the TV training
# subspace_dimension_of_t = 2

# ivector_trainer = bob.learn.em.IVectorTrainer(update_sigma=True)
# ivector_machine = bob.learn.em.IVectorMachine(
#      prior_gmm, subspace_dimension_of_t, 10e-5)
# # train IVector model
# bob.learn.em.train(ivector_trainer, ivector_machine,
#                     gmm_stats_per_class, 500)

# # Printing the session offset w.r.t each Gaussian component
# print(ivector_machine.t)

[ 0.66579325  0.35763949 -0.77270015]
(2, 3)
[-0.00419192  0.31066799 -0.36004278]
(2, 3)
[ 0.13275579  0.05427426  0.00214572]
(2, 3)
[-0.08730011  0.21651309  0.60151869]
(2, 3)
[-0.48253284  0.51413704  0.11431507]
(2, 3)
[ 0.22256881 -0.56830111  0.06756844]
(2, 3)
[ 0.7422685  -0.53990244 -0.98886414]
(2, 3)
[-0.87168615  0.13303508  1.19248367]
(2, 3)
[ 0.56184563  0.83631111  0.04957461]
(2, 3)
[ 0.69899819 -0.13562399  0.30660209]
(2, 3)
[-0.25346344 -0.3098618  -0.17345834]
(2, 3)
[-0.2952284   0.06169462 -0.16099734]
(2, 3)
[-0.119958   -0.26752647  0.05129445]
(2, 3)
[-0.3463939  -0.06795369 -0.27017438]
(2, 3)
[-0.38788667 -0.29786744 -0.36091823]
(2, 3)
[-0.24253953 -0.26782805 -0.13756601]
(2, 3)
[-0.08696947 -0.22948405 -0.20518107]
(2, 3)
[-0.14218116 -0.30797581 -0.058368  ]
(2, 3)
[-0.03155505 -0.15928384  0.27894073]
(2, 3)
[-0.01650821 -0.22245449 -0.27243609]
(2, 3)


In [237]:
a = gmm_stats_per_class[0]

In [239]:
a.sum_pxx

array([[ 0.12095575,  0.03490106,  0.16291825],
       [ 0.32232491,  0.09300494,  0.43414727]])